In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ruptura

In [2]:

components = ruptura.Components()
components.addComponent(
    name="Helium",
    gasPhaseMolFraction=0.98,
    isCarrierGas=True
)
components.addComponent(
    name="nC6",
    gasPhaseMolFraction=0.01,
    isotherms=[["Langmuir", 1.1, 2.14e-4]],
    massTransferCoefficient=0.06,
    axialDispersionCoefficient=0.0
)
components.addComponent(
    name="2MP",
    gasPhaseMolFraction=0.01,
    isotherms=[["Langmuir", 1.4, 1.68e-4]],
    massTransferCoefficient=0.06,
    axialDispersionCoefficient=0.0
)

mix = ruptura.MixturePrediction(
    displayName="CoBDP",
    temperature=443.0, 
    pressureStart=-1.0,
    pressureEnd=-1.0,
    numberOfPressurePoints=100,
    pressureScale="log",
    components=components
)

brk = ruptura.Breakthrough(
    displayName="CoBDP",
    components=components,
    mixturePrediction=mix,
    temperature=443.0,
    numberOfTimeSteps=1e4,
    numberOfGridPoints=100,
    printEvery=1000,
    writeEvery=1,
    totalPressure=2.0e6,
    columnVoidFraction=0.4,
    pressureGradient=0.0,
    columnEntranceVelocity=0.5,
    columnLength=0.3, 
    timeStep=0.0005,
)

Component data
maximum isotherm terms:        1
Component id: 1 [nC6]:
    mol-fraction in the gas:   0.010000 [-]
    mas-transfer coefficient: 0.060000 [1/s]
    diffusion coefficient:     0.000000 [m^2/s]
    number of isotherm sites:  1
    Langmuir isotherm
        q_sat: 1.100000
        b:     0.000214

Component id: 2 [2MP]:
    mol-fraction in the gas:   0.010000 [-]
    mas-transfer coefficient: 0.060000 [1/s]
    diffusion coefficient:     0.000000 [m^2/s]
    number of isotherm sites:  1
    Langmuir isotherm
        q_sat: 1.400000
        b:     0.000168

Component id: 0 [Helium]:
    carrier-gas
    number of isotherm sites:  0
    mol-fraction in the gas:   0.980000 [-]


Column properties
Display-name:                          CoBDP
Temperature:                           443.000000 [K]
Column length:                         0.300000 [m]
Column void-fraction:                  0.400000 [-]
Particle density:                      1000.000000 [kg/m^3]
Total pressure:       

In [3]:
brk.compute()

autosteps 0
Timestep 0, time: 0.000000 [s]
    Average number of mixture-prediction steps: 6.300000


array([[[ 0.00000000e+000,  5.00000000e-001,  2.00000000e+006, ...,
          9.80000000e+001,  0.00000000e+000,  3.88630915e-002],
        [ 3.00000000e-003,  4.99769500e-001,  2.00000000e+006, ...,
          9.99935877e+001,  3.33333333e+006,  0.00000000e+000],
        [ 6.00000000e-003,  4.99754854e-001,  2.00000000e+006, ...,
          1.00000000e+002,  0.00000000e+000,  0.00000000e+000],
        ...,
        [ 2.94000000e-001,  4.99754403e-001,  2.00000000e+006, ...,
          0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
        [ 2.97000000e-001,  4.99754403e-001,  2.00000000e+006, ...,
          9.87971012e-282,  0.00000000e+000,  0.00000000e+000],
        [ 3.00000000e-001,  4.99754403e-001,  2.00000000e+006, ...,
          5.99005189e-320,  0.00000000e+000,  0.00000000e+000]],

       [[ 0.00000000e+000,  5.00000000e-001,  2.00000000e+006, ...,
          1.00000000e+002,  0.00000000e+000,  3.88619256e-002],
        [ 3.00000000e-003,  4.99671977e-001,  2.00000000e+006

In [4]:
brk.data.shape

(10000, 101, 21)

In [5]:
def read_data_to_array(file_path):
    with open(file_path, 'r') as f:
        data = f.read().strip().split('\n\n')  # Split the data into blocks by empty line

    data_blocks = [block.split('\n') for block in data]  # Split each block into lines
    data_lines = [[line.split() for line in block if len(line) and line[0] != "#"] for block in data_blocks]  # Split each line into elements
    
    # Convert everything into integers and reshape to third order array
    data_array = np.array(data_lines, dtype=np.float32)
    
    return data_array

# Use the function
data_array = read_data_to_array('../CoBDP-alkanes-C6/breakthrough/column.data')

In [6]:
data_array.shape

(10000, 101, 21)

In [7]:
np.linalg.norm(brk.data - data_array)

38926116.66955463

In [8]:
(brk.data[0] - data_array[0])[0].argmax()

12